# 支持向量集

如果对SVM的理论不甚了解就去阅读其产品级C++代码，那么读懂的难度很大。但将产品级代码和速度提升部分剥离出去，那么代码就会变得可控，或许这样的代码就可以了。  
有些人认为，<font color=blue size=2>SVM是最好的现成的分类器，这里说的“现成”指的是分类器不加修改即可直接使用</font>。同时，这就意味着在数据上应用基本形式的SVM分类器就可以得到低错误率的结果。SVM能够对训练集之外的数据点做出很好的分类决策。  
SVM有很多实现，但是本章只关注其中最流行的一种实现，即 <font color=blue size=2>序列最小优化(Sequential Minimal ptimization，SMO)算法</font> 。之后，将介绍如何使用一种称为**核函数(kernel)**的方式将SVM扩展到更多数据集上。

## 基于最大间隔分隔数据

|支持向量集||
|:-|:-|
|优点|泛化错误率低，计算开销不大，结果易解释|
|缺点|对参数调节和核函数的选择敏感，原始分类器不加修改仅适用于处理二类问题|
|适用数据类型|数值型和标称型数据|

<center>
    <img src="./images/图6-1 4个线性不可分的数据集.jpg" height='40%' width="40%">
    <br>
    <div style="color:orange; border-bottom: 0.5px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 0.5px;">4个线性不可分的数据集</div>
</center>

**线性可分(linearly separable)与线性不可分**数据。  
将数据集分隔开来的直线称为分隔超平面(separating hyperplane)。由于数据点都在二维平面上，此时分隔超平面就只是一条直线。但是，如果数据集是三维的，那么此时分隔数据的就是一个平面。显而易见，更高维的情况可以依此类推。如果数据集是1024维的，那么就需要一个1023维的某某对象来对数据进行分隔。这个1023维的某某对象到底应该叫什么？N-1维呢？**该对象被称之为超平面(hyperplane)，也就是分类的决策边界**。分布在超平面一侧的所有数据都属于某个类别，而分布在另一侧的所有数据则属于另一个类别。

<center>
    <img src="./images/图6-2 线性可分.jpg" height='40%' width="40%">
    <br>
    <div style="color:orange; border-bottom: 0.5px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 0.5px;">线性可分</div>
</center>

我们希望能采用这种方式来构建分类器，即如果**数据点离决策边界越远，那么其最后的预测结果也就越可信**。我们希望找到离分隔超平面最近的点，确保它们离分隔面的距离尽可能远。这里点到分隔面的距离被称为间隔(margin)。我们**希望间隔尽可能地大**，这是因为如果我们犯错或者在有限数据上训练分类器的话，我们希望分类器尽可能健壮。  
**支持向量(support vector)就是离分隔超平面最近的那些点**。接下来要试着最大化支持向量到分隔面的距离，需要找到此问题的优化求解方法。

## 寻找最大间隔

如何求解数据集的最佳分隔直线？分隔超平面的形式可以写成$w^{T}x+b$。计算点A到分隔超平面的距离，就必须给出点到分隔面的法线或垂线的长度，该值为$|w^{T}A+b|/||w||$。这里的常数b类似于Logistic回归中的截距$w_{0}$。这里的**向量$w$和常数b一起描述了所给数据的分隔线或超平面**。

<center>
    <img src="./images/图6-3 点A到分隔平面的距离就是该点到分割面法线的长度.jpg" height='40%' width="40%">
    <br>
    <div style="color:orange; border-bottom: 0.5px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 0.5px;">点A到分隔平面的距离就是该点到分割面法线的长度</div>
</center>

### 分类器求解的优化问题

**输入数据给分类器会输出一个类别标签，这相当于一个类似于Sigmoid的函数在作用**。下面将使用类似海维赛德阶跃函数(即单位阶跃函数)的函数对$w^{T}x+b$作用得到$f(w^{T}x+b)$，其中当u<0时f(u)输出-1，反之则输出+1。这和Logistic回归有所不同，那里的类别标签是0或1。  
这里的类别标签为什么采用-1和+1，而不是0和1呢？**这是由于-1和+1仅仅相差一个符号，方便数学上的处理**。可以通过一个统一公式来表示间隔或者数据点到分隔超平面的距离，同时不必担心数据到底是属于-1还是+1类。  
当计算数据点到分隔面的距离并确定分隔面的放置位置时，间隔是通过$label*(w^{T}x+b)$来计算，**这时就能体现出-1和+1类的好处了**。如果数据点处于正方向(即+1类)并且离分隔超平面很远的位置时，$w^{T}x+b$会是一个很大的正数，同时$label*(w^{T}x+b)$也会是一个很大的正数。而如果数据点处于负方向(-1类)并且离分隔超平面很远的位置时，此时由于类别标签为-1，则$label*(w^{T}x+b)$仍然是一个很大的正数。

现在的目标就是找出分类器定义中的w和b。为此，我们必须找到具有最小间隔的数据点，而这些数据点也就是前面提到的支持向量。**一旦找到具有最小间隔的数据点，我们就需要对该间隔最大化**。这就可以写作：
$$argmax_{w,b}\left \{  min_{n}(label*(w^{T}x+b)*\frac{1}{||w||})\right \}$$
[直接求解上述问题相当困难](https://blog.csdn.net/arthur503/article/details/25165343),[所以我们将它转换成为另一种更容易求解的形式](https://blog.csdn.net/chaipp0607/article/details/73662441)。令所有支持向量的$label*(w^{T}x+b)$都为1。  
在上述优化问题中，给定了一些约束条件然后求最优值，因此该问题是一个带约束条件的优化问题。这里的约束条件就是$label*(w^{T}x+b)$≥1.0。对于这类优化问题，有一个非常著名的求解方法，即**拉格朗日乘子法**。于是，优化目标函数最后可以写成：
$$max_{a}\left [ \sum_{i=1}^{m}a-\frac{1}{2} \sum_{i,j=1}^{m}label^{(i)}*label^{(j)}*a_{i}*a_{j}\left \langle x_{i}*x_{j} \right \rangle\right ]$$
其约束条件为：
$$a\geqslant 0,\sum_{i=1}^{m}a_{i}*label^{(i)}=0$$

这里有个假设：**数据必须100%线性可分**。目前为止，我们知道几乎所有数据都不不可能那么“干净”。这时我们就可以通过引入所谓松弛变量(slack variable)，来允许有些数据点可以处于分隔面的错误一侧。这样我们的优化目标就能保持仍然不变，但是[此时新的约束条件则变为](https://blog.csdn.net/chaipp0607/article/details/73849539):  
$$c\geqslant a\geqslant 0,\sum_{i=1}^{m}a_{i}*label^{(i)}=0$$
常数C用于控制“最大化间隔”和“保证大部分点的函数间隔小于1.0”这两个目标的权重。在优化算法的实现代码中，常数C是一个参数，因此我们就可以通过调节该参数得到不同的结果。**一旦求出了所有的alpha，那么分隔超平面就可以通过这些alpha来表达**。SVM中的主要工作就是求解这些alpha。  
要理解刚才给出的这些公式还需要大量的知识。如果你有兴趣，我强烈建议去查阅相关的教材以获得上述公式的推导细节。

### SVM应用的一般框架

|SVM的一般流程||
|:-|:-|
|收集数据|可以使用任意方法|
|准备数据|需要数值型数据|
|分析数据|有助于可视化分隔超平面|
|训练算法|SVM的大部分时间都源自训练，该过程主要实现两个参数的调优|
|测试算法|十分简单的计算过程就可以实现|
|使用算法|几乎所有分类问题都可以使用SVM，值得一提的是，SVM本身是一个二类分类器，对多类问题应用SVM需要对代码做一些修改|

## SMO高效优化算法

根据最后两个式子进行优化，其中一个是最小化的目标函数，一个是在优化过程中必须遵循的约束条件。需要做的围绕优化的事情就是训练分类器，一旦得到alpha的最优值，我们就得到了分隔超平面(2维平面中就是直线)并能够将之用于数据分类。

### Platt的SMO算法

SMO表示序列最小优化(Sequential Minimal Optimization)。[Platt](http://xueshu.baidu.com/s?wd=paperuri%3A%289aafb6ef3f7c335a661dca06df2eb6ef%29&filter=sc_long_sign&tn=SE_xueshusource_2kduw22v&sc_vurl=http%3A%2F%2Fciteseerx.ist.psu.edu%2Fviewdoc%2Fdownload%3Bjsessionid%3D09DC6745B2253B96F46E7A9919A26144%3Fdoi%3D10.1.1.42.8725%26rep%3Drep1%26type%3Dpdf&ie=utf-8&sc_us=3346057164966861127)的**SMO算法是将大优化问题分解为多个小优化问题来求解的**。这些小优化问题往往很容易求解，并且对它们进行顺序求解的结果与将它们作为整体来求解的结果是完全一致的。在结果完全相同的同时，SMO算法的求解时间短很多。  
SMO算法的目标是求出一系列alpha和b，一旦求出了这些alpha，就很容易[计算出权重向量w并得到分隔超平面](https://blog.csdn.net/chaipp0607/article/details/73849539)。  
**SMO算法的工作原理**是：每次循环中选择两个alpha进行优化处理。一旦找到一对合适的alpha，那么就增大其中一个同时减小另一个。“合适”指两个alpha必须要符合一定的条件，1.这两个alpha必须要在间隔边界之外，2.这两个alpha还没有进行过区间化处理或者不在边界上。

### 应用简化版SMO算法处理小规模数据集

Platt SMO算法中的外循环确定要优化的最佳alpha对。而简化版却会跳过这一部分，首先在数据集上遍历每一个alpha，然后在剩下的alpha集合中随机选择另一个alpha，从而构建alpha对。有一点相当重要，就是**要同时改变两个alpha**。之所以这样做是因为有一个约束条件：
$$\sum a_{i}*label^{(i)}=0$$
**由于改变一个alpha可能会导致该约束条件失效，因此总是同时改变两个alpha**。  
为此，将构建一个辅助函数，用于在某个区间范围内随机选择一个整数。同时，也需要另一个辅助函数，用于在数值太大时对其进行调整。

In [2]:
# SMO算法中的辅助函数
import numpy as np


def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat


def selectJrand(i, m):
    j = i  # we want to select any J not equal to i
    while (j == i):
        j = int(np.random.uniform(0, m))
    return j


def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

loadDatSet()函数，该函数打开文件并对其进行逐行解析，从而得到每行的类标签和整个数据矩阵。  
selectJrand()有两个参数值，其中i是第一个alpha的下标，m是所有alpha的数目。只要函数值不等于输入值i，函数就会进行随机选择。  
clipAlpha()，它是用于调整大于H或小于L的alpha值。

In [3]:
dataArr, labelArr = loadDataSet('testSet.txt')
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,


limit_output extension: Maximum message size of 100 exceeded with 653 characters

**[SMO算法](https://blog.csdn.net/luoshixian099/article/details/51227754)的第一个版本伪代码**：  
　创建一个alpha向量并将其初始化为0向量  
　当迭代次数小于最大迭代次数时(外循环)  
　　对数据集中的每个数据向量(内循环)：  
　　　如果该数据向量可以被优化：  
　　　　随机选择另外一个数据向量  
　　　　同时优化这两个向量  
　　　　如果两个向量都不能被优化，退出内循环  
　　如果所有向量都没被优化，增加迭代数目，继续下一次循环

In [4]:
# 简化版SMO算法
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    b = 0
    m, n = np.shape(dataMatrix)
    alphas = np.mat(np.zeros((m, 1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(np.multiply(alphas, labelMat).T*dataMatrix*dataMatrix[i, :].T) + b
            # if checks if an example violates KKT conditions
            Ei = fXi - float(labelMat[i])
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i, m)#随机选择第二个alpha
                fXj = float(np.multiply(alphas, labelMat).T *
                            (dataMatrix*dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                #保证alpha在0到C之间
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H:
                    print("L==H")
                    continue
                eta = 2.0 * dataMatrix[i, :]*dataMatrix[j, :].T - dataMatrix[i,
                                    :]*dataMatrix[i, :].T - dataMatrix[j, :]*dataMatrix[j, :].T
                if eta >= 0:
                    print("eta>=0")
                    continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough")
                    continue
                # update i by the same amount as j对i进行修改，修改量与j相同
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])
                # the update is in the oppostie direction但方向相反
                b1 = b - Ei - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i, :]*dataMatrix[i,
                                   :].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i, :]*dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i, :]*dataMatrix[j,
                                   :].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j, :]*dataMatrix[j, :].T
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print("iter: %d i:%d, pairs changed %d" %
                      (iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0):
            iter += 1
        else:
            iter = 0
        print("iteration number: %d" % iter)
    return b, alphas

smoSimple()有5个输入参数：数据集、类别标签、常数C、容错率和退出前最大的循环次数。alpha列矩阵，矩阵中元素都初始化为0。iter变量，**存储的则是在没有任何alpha改变的情况下遍历数据集的次数**。  
变量alphaPairsChanged用于记录alpha是否已经进行优化。[fXi是我们预测的类别。计算误差Ei](https://www.cnblogs.com/pinard/p/6111471.html)。如果误差很大，那么可以对alpha值进行优化。if语句中，正负间隔都会被测试。同时检查alpha值，以保证其不能等于0或C。由于后面alpha小于0或大于C时将被调整为0或C，所以**一旦在该if语句中它们等于这两个值的话，那么它们就已经在“边界”上了，因而不再能够减小或增大，因此也就不值得再对它们进行优化了**。  
随机选择第二个alpha值，即alpha[j]。Python则会通过引用的方式传递所有列表。之后**计算L和H，它们用于将alpha[j]调整到0到C之间**。如果L和H相等，就不做任何改变，直接执行continue语句。  
**Eta是alpha[j]的最优修改量**。如果eta为0，那么计算新的alpha[j]就比较麻烦了。有需要的读者可以阅读Platt的原文来了解更多的细节。现实中，这种情况并不常发生，因此忽略这一部分通常也无伤大雅。于是，可以计算出一个新的alpha[j]，然后利用L与H值对其进行调整。  
然后，就是需要**检查alpha[j]是否有轻微改变**。如果是的话，就退出for循环。然后，alpha[i]和alpha[j]同样进行改变，虽然改变的大小一样，但是改变的方向正好相反（即如果一个增加，那么另外一个减少）。在对alpha[i]和alpha[j]进行优化之后，给这两个alpha值设置一个常数项b。
在for循环之外，需要检查alpha值是否做了更新，如果有更新则将iter设为0后继续运行程序。**只有在所有数据集上遍历maxIter次，且不再发生任何alpha修改之后**，程序才会停止并退出while循环。

In [5]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

iter: 0 i:0, pairs changed 1
iter: 0 i:3, pairs changed 2
L==H
iter: 0 i:10, pairs changed 3
iter: 0

limit_output extension: Maximum message size of 100 exceeded with 3837 characters

我们可以直接观察alpha矩阵本身，但是**其中的零元素太多**。  
由于**SMO算法的随机性**，每次运行后结果可能会与下述结果不同。alphas[alphas>0]命令是数组过滤(array filtering)的一个实例，而且它只对NumPy类型有用，却并不适用于Python中的正则表(regular list)。

In [6]:
b, alphas[alphas > 0]

(matrix([[-3.77228363]]),
 matrix([[0.05516594, 0.30016951, 0.08866521, 0.26667024]]))

为了得到支持向量的个数

In [7]:
np.shape(alphas[alphas > 0])

(1, 4)

为了解哪些数据点是支持向量

In [8]:
for i in range(100):
    if alphas[i] > 0.0:
        print(dataArr[i], labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[5.286862, -2.358286] 1.0
[6.080573, 0.418886] 

limit_output extension: Maximum message size of 100 exceeded with 104 characters

<center>
    <img src="./images/图6-4 简化smo算法结果，画圈的为支持向量.jpg" height='40%' width="40%">
    <br>
    <div style="color:orange; border-bottom: 0.5px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 0.5px;">简化smo算法结果，画圈的为支持向量</div>
</center>

这只是一个仅有100个点的小规模数据集。在更大的数据集上，收敛时间会变得更长。下面，将通过**构建完整SMO算法来加快其运行速度**。

## 利用完整Platt SMO算法加速优化

实现alpha的更改和代数运算的优化环节一模一样。在优化过程中，唯一的不同就是选择alpha的方式。完整版的**Platt SMO算法应用了一些能够提速的启发方法**。  
Platt SMO算法是通过一个外循环来选择第一个alpha值的，并且其选择过程会在两种方式之间进行交替：**1.在所有数据集上进行单遍扫描，2.在非边界alpha中实现单遍扫描**。而所谓非边界alpha指的就是那些不等于边界0或C的alpha值。----对整个数据集的扫描相当容易，而实现非边界alpha值的扫描时，首先需要建立这些alpha值的列表，然后再对这个表进行遍历。同时，该步骤会跳过那些已知的不会改变的alpha值。  
在选择第一个alpha值后，算法会通过一个内循环来选择第二个alpha值。在优化过程中，会通过最大化步长的方式来获得第二个alpha值。在简化版SMO算法中，会在选择j之后计算错误率Ej。但在这里，会建立一个全局的缓存用于保存误差值，并**从中选择使得步长或者说Ei-Ej最大的alpha值**。  
下面包含1个用于清理代码的数据结构和3个用于对E进行缓存的辅助函数。

In [9]:
# 完整版Platt SMO的支持函数
class optStruct:
    # Initialize the structure with the parameters
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        # first column is valid flag
        self.eCache = np.mat(np.zeros((self.m, 2)))
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)


def calcEk(oS, k):
    fXk = float(np.multiply(oS.alphas, oS.labelMat).T*oS.K[:, k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek


def selectJ(i, oS, Ei):  # this is the second choice -heurstic, and calcs Ej
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    # set valid #choose the alpha that gives the maximum delta E
    oS.eCache[i] = [1, Ei]
    validEcacheList = np.nonzero(oS.eCache[:, 0].A)[0]  # 返回非0下标的索引tuple几个维度几个array
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:  # loop through valid Ecache values and find the one that maximizes delta E
            if k == i:
                continue  # don't calc for i, waste of time
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:  # in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej


def updateEk(oS, k):  # after any alpha has changed update the new value in the cache
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]

建立一个数据结构来保存所有的重要值，而这个过程可以通过一个对象来完成。这里使用对象的目的并不是为了面向对象的编程，而只是作为一个数据结构来使用对象。为达到这个目的，需要构建一个仅包含init方法的optStruct类。**eCache的第一列给出的是eCache是否有效的标志位，而第二列给出的是实际的E值**。  
对于给定的alpha值，辅助函数calcEk()能够计算E值并返回。以前，该过程是采用**内嵌**的方式来完成的，但是由于该过程在这个版本的SMO算法中出现**频繁**，这里必须要将其单独拎出来。  
函数selectJ()用于选择第二个alpha或者说内循环的alpha值。目标是选择合适的第二个alpha值以**保证在每次优化中采用最大步长**。该函数的误差值与第一个alpha值Ei和下标i有关。首先将输入值Ei在缓存中设置成为有效的。这里的有效(valid)意味着它已经计算好了。程序会在所有的值上进行循环并**选择其中使得改变最大的那个值**。如果这是第一次循环的话，那么就随机选择一个alpha值。  
updateEk()，它会**计算误差值并存入缓存当中**。在对alpha值进行优化之后会用到这个值。  
[nonzero()](https://www.cnblogs.com/1zhk/articles/4782812.html)

In [10]:
# 完整Platt SMO算法中的优化例程
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j, Ej = selectJ(i, oS, Ei)  # this has been changed from selectJrand第二个alpha选择中的启发式方法
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L==H")
            return 0
        eta = 2.0 * oS.K[i, j] - oS.K[i, i] - oS.K[j, j]  # changed for kernel
        if eta >= 0:
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)  # added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i] * \
            (alphaJold - oS.alphas[j])  # update i by the same amount as j
        # added this for the Ecache                    #the update is in the oppostie direction
        updateEk(oS, i)
        b1 = oS.b - Ei - oS.labelMat[i]*(oS.alphas[i]-alphaIold) * \
            oS.K[i, i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i, j]
        b2 = oS.b - Ej - oS.labelMat[i]*(oS.alphas[i]-alphaIold) * \
            oS.K[i, j] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j, j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2.0
        return 1
    else:
        return 0

代码几乎和smoSimple()函数一模一样，但是这里的代码已经使用了自己的数据结构。该结构在参数oS中传递。第二个重要的修改就是使用程序清单6-3中的selectJ()而不是selectJrand()来选择第二个alpha的值。最后，在alpha值改变时更新Ecache。

In [11]:
# 完整版Platt SMO的外循环代码
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):  # full Platt SMO
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:  # go over all
            for i in range(oS.m):
                alphaPairsChanged += innerL(i, oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" %
                      (iter, i, alphaPairsChanged))
            iter += 1
        else:  # go over non-bound (railed) alphas
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" %
                      (iter, i, alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False  # toggle entire set loop
        elif (alphaPairsChanged == 0):
            entireSet = True
        print("iteration number: %d" % iter)
    return oS.b, oS.alphas

当迭代次数超过指定的最大值，或者遍历整个集合都未对任意alpha对进行修改时，就退出循环。这里maxIter变量和函数smoSimple()中的作用有一点不同，后者当没有任何alpha发生改变时会将整个集合的一次遍历过程计成一次迭代，而这里的一次迭代定义为一次循环过程，而不管该循环具体做了什么事。**此时，如果在优化过程中存在波动就会停止，因此这里的做法优于smoSimple()函数中的计数方法**。  
while循环的内部与smoSimple()中有所不同，**一开始的for循环在数据集上遍历任意可能的alpha**。我们通过调用**innerL()来选择第二个alpha，并在可能时对其进行优化处理**。如果有任意一对alpha值发生改变，那么会返回1。**第二个for循环遍历所有的非边界alpha值**，也就是不在边界0或C上的值。接下来，对for循环在非边界循环和完整遍历之间进行切换，并打印出迭代次数。最后程序将会返回常数b和alpha值。

In [14]:
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

L==H
fullSet, iter: 0 i:0, pairs changed 0
L==H
fullSet, iter: 0 i:1, pairs changed 0
fullSet, iter:

limit_output extension: Maximum message size of 100 exceeded with 8766 characters

可以检查b和多个alpha的值。运行10次算法，求平均值，最后得到的结果是0.78秒。而在同样的数据集上，smoSimple()函数平均需要14.5秒。在更大规模的数据集上结果可能更好，另外也存在很多方法可以进一步提升其运行速度。  
**如果修改容错值结果会怎样？如果改变C的值又如何呢？**常数C给出的是不同优化问题的权重。常数C一方面要保障所有样例的间隔不小于1.0，另一方面又要使得分类间隔要尽可能大，并且要在这两方面之间平衡。如果C很大，那么分类器将力图通过分隔超平面对所有的样例都正确分类。

<center>
    <img src="./images/图6-5 运行完整SMO算法后得到的支持向量.jpg" height='40%' width="40%">
    <br>
    <div style="color:orange; border-bottom: 0.5px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 0.5px;">运行完整SMO算法后得到的支持向量</div>
</center>

支持向量更多。因为简化版算法通过随机的方式选择alpha对的。这种简单的方式也可以工作，但是效果却不如完整版本好，后者覆盖了整个数据集。读者可能还认为选出的支持向量应该始终最接近分隔超平面。给定C的设置，图中画圈的支持向量就给出了满足算法的一种解。如果数据集非线性可分，就会发现支持向量会在超平面附近聚集成团。

如何利用它们进行分类呢？首先必须基于alpha值得到超平面，这也包括了w的计算。下面列出的一个小函数可以用于实现上述任务：

In [12]:
def calcWs(alphas, dataArr, classLabels):
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).transpose()
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w += np.multiply(alphas[i]*labelMat[i], X[i, :].T)
    return w

循环中实现多个数的乘积。看一下前面计算出的任何一个alpha，就不会忘记大部分alpha值为0。**而非零alpha所对应的也就是支持向量**。虽然上述for循环遍历了数据集中的所有数据，但是最终起作用的只有支持向量。由于对w计算毫无作用，所以数据集的其他数据点也就会很容易地被舍弃。

In [15]:
ws = calcWs(alphas, dataArr, labelArr)
ws
dataMat = np.mat(dataArr)
dataMat[0]*np.mat(ws)+b
labelArr[0]

array([[ 0.65307162],
       [-0.17196128]])

matrix([[-0.92555695]])

-1.0

如果该值大于0，那么其属于1类；如果该值小于0，那么则属于-1类。对于数据点0，应该得到的类别标签是-1

值得注意的是，这里两个类中的数据点分布在一条直线的两边。倘若两类数据点分别分布在一个圆的内部和外部，那么会得到什么样的分类面呢？下面介绍一种方法对分类器进行修改，以说明类别区域形状不同情况下的数据集分隔问题。

## 在复杂数据上应用核函数

<center>
    <img src="./images/图6-6 这里存在分隔方形点和圆形点的模式.jpg" height='40%' width="40%">
    <br>
    <div style="color:orange; border-bottom: 0.5px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 0.5px;">这里存在分隔方形点和圆形点的模式</div>
</center>

这类数据来描述非线性可分的情况。显而易见，在该数据中存在某种可以识别的模式。其中一个问题就是，能否像线性情况一样，利用强大的工具来捕捉数据中的这种模式？显然，答案是肯定的。使用一种称为**核函数(kernel)的工具将数据转换成易于分类器理解的形式**。本节首先解释核函数的概念，并介绍它们在支持向量机中的使用方法。然后，介绍一种称为径向基函数(radial basis function)的最流行的核函数。最后，将该核函数应用于我们前面得到的分类器。

### 利用核函数将数据映射到高维空间

数据点处于一个圆中，人类的大脑能够意识到这一点。然而，对于分类器而言，它只能识别分类器的结果是大于0还是小于0。或许可以对圆中的数据进行某种形式的转换，从而得到某些新的变量来表示数据。在这种表示情况下，我们就更容易得到大于0或者小于0的测试结果。在这个例子中，**将数据从一个特征空间转换到另一个特征空间**。在新空间下，我们可以很容易利用已有的工具对数据进行处理。**称为从一个特征空间到另一个特征空间的映射**。在通常情况下，这种映射会将低维特征空间映射到高维空间。

这种从某个特征空间到另一个特征空间的映射是通过核函数来实现的。读者可以把核函数想象成一个包装器(wrapper)或者是接口(interface)，它能把数据从某个很难处理的形式转换成为另一个较容易处理的形式。**如果上述特征空间映射的说法听起来很让人迷糊的话，那么可以将它想象成为另外一种距离计算的方法**。距离计算的方法有很多种，核函数一样具有多种类型。经过空间转换之后，**可以在高维空间中解决线性问题，这也就等价于在低维空间中解决非线性问题。**

**SVM优化中一个特别好的地方就是，所有的运算都可以写成内积(inner product，也称点积)的形式**。向量的内积指的是两个向量相乘，之后得到单个标量或者数值。我们可以把内积运算替换成核函数，而不必做简化处理。将内积替换成核函数的方式被称为核技巧(kernel trick)或者核“变电”(kernel substation)。

核函数并不仅仅应用于支持向量机，**很多其他的机器学习算法也都用到核函数**。接下来介绍一个流行的核函数，那就是径向基核函数。

### 径向基核函数

径向基函数是一个采用向量作为自变量的函数，能够基于向量距离运算输出一个标量。这个距离可以是从<0,0>向量或者其他向量开始计算的距离。径向基函数的高斯版本，其具体公式为：
$$k(x,y)=exp\left ( \frac{-\left \| x-y \right \|^{2}}{2\sigma ^{2}} \right )$$
其中，**$\sigma$是用户定义的用于确定到达率(reach)或者说函数值跌落到0的速度参数**。

**上述高斯核函数将数据从其特征空间映射到更高维的空间，具体来说这里是映射到一个无穷维的空间**。高斯核函数只是一个常用的核函数，使用者并不需要确切地理解数据到底是如何表现的，而且使用高斯核函数还会得到一个理想的结果。(在上面的例子中，数据点基本上都在一个圆内。对于这个例子，我们可以直接检查原始数据，并意识到只要度量数据点到圆心的距离即可。然而，如果碰到了一个不是这种形式的新数据集，那么我们就会陷入困境。)**在该数据集上，使用高斯核函数可以得到很好的结果**。当然，该函数也可以用于许多其他的数据集，并且也能得到低错误率的结果。

In [13]:
# 核转换函数
def kernelTrans(X, A, kTup):  # calc the kernel or transform data to a higher dimensional space
    m, n = np.shape(X)
    K = np.mat(np.zeros((m, 1)))
    if kTup[0] == 'lin':
        K = X * A.T  # linear kernel,machine-learning P128 by 周志华
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow*deltaRow.T
        # divide in NumPy is element-wise not matrix like Matlab
        K = np.exp(K/(-1*kTup[1]**2))
    else:
        raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

**全局的K值只需计算一次。**然后，当想要使用核函数时，就可以对它进行调用。这也省去了很多冗余的计算开销。

计算矩阵K时，多次调用了函数kernelTrans()。该函数有3个输入参数：2个数值型变量和1个元组。**元组kTup给出的是核函数的信息**。元组的第一个参数是描述所用核函数类型的一个字符串，其他2个参数则都是核函数可能需要的可选参数。  
**在线性核函数的情况下**，内积计算在“所有数据集”和“数据集中的一行”这两个输入之间展开。**在径向基核函数的情况下**，在for循环中对于矩阵的每个元素计算高斯函数的值。而在for循环结束之后，我们将计算过程应用到整个向量上去。  
最后，如果遇到一个无法识别的元组，程序就会抛出异常，因为在这种情况下不希望程序再继续运行，这一点相当重要。

### 在测试中使用核函数

径向基函数有一个用户定义的输入$\sigma$。首先，我们需要确定它的大小，然后利用该核函数构建出一个分类器。

In [16]:
# 利用核函数进行分类的径向基测试函数
def testRbf(k1=1.3):
    dataArr, labelArr = loadDataSet('testSetRBF.txt')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))  # C=200 important
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]  # get matrix of only support vectors
    labelSV = labelMat[svInd]#labels of support vectors
    print("there are %d Support Vectors" % np.shape(sVs)[0])
    m, n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr, labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    m, n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ('rbf', k1))
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the test error rate is: %f" % (float(errorCount)/m))

输入参数是高斯径向基函数中的一个用户定义变量。  
优化过程结束后，在后面的矩阵数学运算中建立了数据的矩阵副本，并且找出那些**非零的alpha值，从而得到所需要的支持向量**；同时，也就得到了这些支持向量和alpha的类别标签值。这些值仅仅是需要分类的值。  
最重要的是for循环开始的那两行，它们给出了如何利用核函数进行分类。首先利用结构初始化方法中使用过的kernelTrans()函数，得到转换后的数据。然后，再用其与前面的alpha及类别标签值求积。**其中需要特别注意的另一件事是，在这几行代码中，是如何做到只需要支持向量数据就可以进行分类的。除此之外，其他数据都可以直接舍弃。**  
第二个for循环采用的测试数据集。可以比较不同的设置在测试集和训练集上表现出的性能。

In [17]:
testRbf()

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pa

limit_output extension: Maximum message size of 100 exceeded with 10058 characters

<center>
    <img src="./images/图6-7 参数k1=0.1时的径向基函数。该参数此时减少了每个支持向量的影响程度，因此需要更多的支持向量.jpg" height='40%' width="40%">
    <br>
    <div style="color:orange; border-bottom: 0.5px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 0.5px;">参数k1=0.1时的径向基函数.该参数此时减少了每个支持向量的影响程度,因此需要更多的支持向量</div>
</center>

100个数据点，其中的85个为支持向量。优化算法发现，必须使用这些支持向量才能对数据进行正确分类。这就可能给了读者径向基函数到达率太小的直觉。通过增加$\sigma$来观察错误率的变化情况。

<center>
    <img src="./images/图6-8 k1=1.3时的径向基函数。支持向量在决策边界聚集.jpg" height='40%' width="40%">
    <br>
    <div style="color:orange; border-bottom: 0.5px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 0.5px;">k1=1.3时的径向基函数. 支持向量在决策边界聚集</div>
</center>

27个支持向量，其数目少了很多。观察testRbf()的输出结果就会发现，此时的测试错误率也在下降。**该数据集在这个设置的某处存在着最优值。**如果降低$\sigma$，那么训练错误率就会降低，但是测试错误率却会上升。  
**支持向量的数目存在一个最优值。**SVM的优点在于它能对数据进行高效分类。如果支持向量太少，就可能会得到一个很差的决策边界(下个例子会说明这一点)；如果支持向量太多，也就相当于每次都利用整个数据集进行分类，这种分类方法称为k近邻。

## 示例：手写识别问题回顾

k近邻占用的内存太大了。必须在保持其性能不变的同时，使用更少的内存。kNN方法效果不错，但是需要保留所有的训练样本。而对于支持向量机而言，其需要保留的样本少了很多**(即只保留支持向量)**，但是能获得可比的效果。

||示例：基于SVM的数字识别|
|:-|:-|
|收集数据|提供的文本文件|
|准备数据|基于二值图像构造向量|
|分析数据|对图像向量进行目测|
|训练算法|采用两种不同的核函数，并对径向基核函数采用不同的设置来运行SMO算法|
|测试算法|编写一个函数来测试不同的核函数并计算错误率|
|使用算法|一个图像识别的完整应用还需要一些图像处理的知识，这里并不打算深入介绍|

In [18]:
# 基于SVM的手写数字识别
def img2vector(filename):
    returnVect = np.zeros((1, 1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])
    return returnVect


def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)  # load the training set
    m = len(trainingFileList)
    trainingMat = np.zeros((m, 1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]  # take off .txt
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9:
            hwLabels.append(-1)
        else:
            hwLabels.append(1)
        trainingMat[i, :] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels


def testDigits(kTup=('rbf', 10)):
    dataArr, labelArr = loadImages('trainingDigits')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    svInd = np.nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print("there are %d Support Vectors" % np.shape(sVs)[0])
    m, n = np.shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr, labelArr = loadImages('testDigits')
    errorCount = 0
    datMat = np.mat(dataArr)
    labelMat = np.mat(labelArr).transpose()
    m, n = np.shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], kTup)
        predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b
        if np.sign(predict) != np.sign(labelArr[i]):
            errorCount += 1
    print("the test error rate is: %f" % (float(errorCount)/m))

函数loadImages()同支持向量机一起使用时，类别标签为-1或者+1。本质上，支持向量机是一个二类分类器，其分类结果不是+1就是-1。[基于SVM构建多类分类器已有很多研究和对比了](https://ieeexplore.ieee.org/ielx5/72/21990/01021904.pdf?tp=&arnumber=1021904&isnumber=21990&tag=1)。  
testDigits()函数元组kTup是输入参数。

In [19]:
testDigits()

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pa

limit_output extension: Maximum message size of 100 exceeded with 3826 characters

|内核，设置|训练错误率%|测试错误率5|支持向量数|
|:-|:-|:-|:-|
|RBF,0.1|0|52|402|
|RBF,0.1|0|3.2|402|
|RBF,0.1|0|0.5|99|
|RBF,0.1|0.2|2.2|41|
|RBF,0.1|4.5|4.3|26|
|Linear|2.7|2.2|38|

前面$\sigma$最优在1.3左右，这里在10左右，为什么差距这么大？**原因在于数据的不同**。(最优参数只能多次尝试得出)  
注意，**最小的训练错误率并不对应于最小的支持向量数目**。另一个值得注意的就是，**线性核函数的效果并不是特别的糟糕**。可以以牺牲线性核函数的错误率来换取分类速度的提高。尽管这一点在实际中是可以接受的，但是还得取决于具体的应用。

## 本章小结

支持向量机是一种分类器。之所以称为“机”是因为它会产生一个二值决策结果，即它是一种决策“机”。**支持向量机的泛化错误率较低**，也就是说它具有良好的学习能力，且学到的结果具有很好的推广性。这些优点使得支持向量机十分流行，有些人认为它是监督学习中最好的定式算法。

支持向量机试图通过求解一个二次优化问题来最大化分类间隔。过去，训练支持向量机常采用非常复杂并且低效的二次规划求解方法。John Platt引入了**SMO算法，此算法可以通过每次只优化2个alpha值来加快SVM的训练速度**。完整版算法不仅大大地提高了优化的速度，还使其存在一些[进一步提高运行速度的空间](https://www.researchgate.net/publication/2446719_Improvements_to_Platts_SMO_Algorithm_for_SVM_classifier_design)。

核方法或者说核技巧会将数据(有时是非线性数据)从一个低维空间映射到一个高维空间，可以**将一个在低维空间中的非线性问题转换成高维空间下的线性问题来求解**。核方法不止在SVM中适用，还可以用于其他算法中。而其中的**径向基函数是一个常用的度量两个向量距离的核函数**。

支持向量机是一个二类分类器。当用其解决多类问题时，则需要额外的方法对其进行扩展。**SVM的效果也对优化参数和所用核函数中的参数敏感。**